In [1]:
import gensim.downloader

In [2]:
glove_vectors = gensim.downloader.load('glove-twitter-50')

In [27]:
glove_vectors.most_similar('voice')

[('song', 0.8404531478881836),
 ('music', 0.7873961329460144),
 ('singing', 0.7774211764335632),
 ('kelly', 0.7702898979187012),
 ('lyrics', 0.7667415142059326),
 ('beyonce', 0.7661986351013184),
 ('sound', 0.7640264630317688),
 ('makes', 0.762234091758728),
 ('chris', 0.7618743777275085),
 ('bad', 0.7614580988883972)]

In [18]:
from gensim.parsing.preprocessing import STOPWORDS
print(STOPWORDS)



frozenset({'what', 'just', 'please', 'hasnt', 'beside', 'across', 'various', 'next', 'except', 'de', 'two', 'again', 'could', 'seemed', 'make', 'made', 'well', 'was', 'whenever', 'anywhere', 'herself', 'something', 'anything', 'say', 'but', 'became', 'whatever', 'yet', 'fire', 'your', 'on', 'nowhere', 'us', 'here', 'ourselves', 'more', 'hereafter', 'therein', 'they', 'how', 'does', 'do', 'call', 'if', 'have', 'must', 'by', 'his', 'at', 'however', 'had', 'twelve', 'itself', 'used', 'their', 'computer', 'is', 'ie', 'might', 'part', 'whence', 'amongst', 'take', 'anyone', 'whom', 'become', 'less', 'not', 'when', 'eleven', 'see', 'such', 'beyond', 'of', 'the', 'ten', 'thru', 'can', 'yourselves', 'seem', 'almost', 'herein', 'cant', 'along', 'through', 'myself', 'kg', 'whither', 'although', 'cannot', 'without', 'elsewhere', 'around', 'she', 'cry', 'much', 'until', 'eight', 'neither', 'sometime', 'some', 'bill', 'down', 'whose', 'everywhere', 'go', 'three', 'where', 'themselves', 'enough', 'fo

In [20]:
l = ['1' ,'2' , '3']
m = ['2' , '4']
p = frozenset(l)
p1 = frozenset(m)
print(p1.difference(p))
ll = list(p)
print(ll)

frozenset({'4'})
['1', '2', '3']


In [26]:
from flask import Flask, request, render_template

app = Flask(__name__)


@app.route('/')
def my_form():
    return render_template('my-form.html')


@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    number = request.form['number']

    list_of_words = text.split()

    hashtags_list = []
    most_similar_vec = []

    for word in list_of_words:
        hashtags_list.append(word.lower())
        if glove_vectors.__contains__(word):

            most_similar_vec = glove_vectors.most_similar(word)
            count = 0

            for word_1, similarity in most_similar_vec:

                hashtags_list.append(word_1)
                count = count + 1

    # finding the cosine similarity of the generated words with the entire entered text and sorting it in decending order w.r.t similarity

    final_list = []
    for hash_word in hashtags_list:
        cosine_sim = 0
        # not including words with length less or equal to 4
        if len(hash_word) <= 3:
            continue
        for word in text:

            # checking if the input word is in the vocabulary
            if glove_vectors.__contains__(word) and glove_vectors.__contains__(hash_word):
                # comparing each word of the generated word with the entire text
                cosine_sim = cosine_sim + \
                    glove_vectors.similarity(word, hash_word)

        final_list.append((hash_word, cosine_sim))

    # sorting with respect to total cosine similarity in increasing order
    # words having high total cosine similarity are not good hashtags

    final_list.sort(key=lambda x: x[1])

    count = 0
    # removing duplicates
    newlist = sorted(set(final_list), key=lambda x: final_list.index(x))

    #extracting the words from pair of word , similarity in newlist (making list of string from list of pair of string , float)

    newlist1 = [] 

    for word , similarity in newlist:
        newlist1.append(word) 
    
    #removing the stop words from the newlist 

    #converting the newlist to frozenset 

    newlist1_frozen = frozenset(newlist1)

    #taking the difference from the stopwords

    newlist1_frozen = newlist1_frozen.difference(STOPWORDS)

    # coverting the set back to the list 

    newlist1 = list(newlist1_frozen)
    processed_text = ""

    ct = 0

    for word in newlist1:

        processed_text = processed_text+'#'+word + ' '

        ct = ct + 1

        if str(ct) == number:
            break

    return render_template('result.html', result=processed_text)


app.run(host='localhost', port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Mar/2022 22:06:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2022 22:06:36] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2022 22:07:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2022 22:07:18] "POST / HTTP/1.1" 200 -
